In [ ]:
# Google Generative AI 패키지를 설치하여 API에 접근할 수 있도록 합니다.
!pip install -q -U google-generativeai==0.8.3

In [ ]:
# 필요한 모듈을 임포트합니다.
import google.generativeai as genai
import pickle
from tqdm import tqdm
import re
# Generative AI 모델을 설정합니다. "gemini-1.5-flash" 모델을 사용하여 질문과 답변을 생성
genai.configure(api_key="GOOGLE_API_KEY")
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
# 청킹된 데이터 파일인 '전자금융거래.pk' 파일에서 데이터를 불러옴
data_list = pickle.load(open('전자금융거래.pk', 'rb'))

In [ ]:
# 질문과 답변을 저장할 리스트를 생성
qa_data = []

# AI 모델에 전달할 프롬프트를 정의
prompt = """
너는 지문을 보고 사용자가 궁금해할만한 질문과 답변을 만드는 사람이야.
- 질문과 답변을 5개씩 만들어줘.
질문1:
답변1:
형식으로 답변해줘야해
- 각 질문과 답변의 내용은 다양한 주제를 포함하도록 만들어줘
- 각 질문과 답변은 아래 예시 형식으로 작성해줘.
   그 외에 텍스트는 사용하지 마.

   - 예시
질문1: ~~~
답변1: ~~~
... 생략 ...
질문5: ~~~
답변5: ~~~

    - 지문
{chunk}
"""

# 각 텍스트 청크에 대해 질문과 답변을 생성하고, qa_data에 저장
for i, chunk in enumerate(tqdm(data_list)):
    try:
        # 각 청크를 프롬프트에 삽입하여 질문과 답변을 생성
        model_output = model.generate_content(prompt.format(chunk=chunk)).text
    except Exception as e:
        print(e)
        continue

    # 정규표현식을 사용하여 생성된 텍스트에서 질문과 답변을 추출
    questions = re.findall(r"질문\d+:\s*(.*?)\s*답변\d+:", model_output, re.DOTALL)
    answers = re.findall(r"답변\d+:\s*(.*?)(?:\n질문|$)", model_output, re.DOTALL)

    # 질문과 답변을 쌍으로 묶어 qa_data에 추가
    for question, answer in zip(questions, answers):
        question, answer = question.strip(), answer.strip()
        qa_data.append({
            "index": i,
            "chunk": chunk,
            "question": question,
            "answer": answer
        })
        print(f"Q: {question}\nA: {answer}")

# 생성된 QA 데이터를 pickle 모듈을 사용하여 바이너리 파일로 저장
with open('qa_data.pkl', 'wb') as f:
    pickle.dump(qa_data, f)

  1%|▏         | 1/75 [00:05<06:14,  5.06s/it]

Q: 전자금융거래법의 목적은 무엇인가요?
A: 전자금융거래법은 전자금융거래의 안전성과 신뢰성을 확보하고 전자금융업의 건전한 발전을 위한 기반을 조성하여 국민의 금융 편의를 증진하고 국민경제 발전에 기여하는 것을 목적으로 합니다.
Q: 전자금융거래법에서 정의하는 "전자금융거래"란 무엇인가요?
A: 전자금융거래는 금융회사 또는 전자금융업자가 전자적 장치를 통해 금융상품 및 서비스를 제공하고, 이용자가 직접 대면하거나 의사소통 없이 자동화된 방식으로 이용하는 거래를 말합니다.
Q: 전자금융거래법에서 정의하는 "전자지급거래"란 무엇인가요?
A: 전자지급거래는 지급인이 금융회사 또는 전자금융업자를 통해 전자지급수단을 이용하여 수취인에게 자금을 이동시키는 전자금융거래를 말합니다.
Q: 전자금융거래법에서 정의하는 "금융회사"에는 어떤 기관들이 포함되나요?
A: 전자금융거래법에서 정의하는 "금융회사"는 금융위원회의 설치 등에 관한 법률에 따른 은행, 증권사, 보험사, 금융투자회사, 신용정보회사, 여신전문금융업법에 따른 여신전문금융회사, 우체국예금·보험에 관한 법률에 따른 체신관서 등을 포함합니다.
Q: 전자금융거래법은 언제부터 시행되었나요?
A: 전자금융거래법은 2020년 12월 10일부터 시행되었습니다.


  3%|▎         | 2/75 [00:09<05:40,  4.67s/it]

Q: 전자금융업자는 어떤 기관인가요?
A: 전자금융업자는 「전자금융거래법」 제28조에 따라 허가를 받거나 등록을 한 자로서, 금융회사를 제외한 기관입니다. 즉, 금융회사가 아닌 기관이 전자금융거래를 할 수 있도록 허가를 받은 경우를 말합니다.
Q: 전자금융보조업자는 어떤 업무를 하는가요?
A: 전자금융보조업자는 금융회사 또는 전자금융업자를 위해 전자금융거래를 보조하거나 그 일부를 대행하는 업무를 합니다. 예를 들어, 결제대행 서비스를 제공하거나 전자금융거래 시스템을 운영하는 등의 업무를 수행합니다.
Q: 전자금융거래계약은 무엇을 의미하나요?
A: 전자금융거래계약은 전자금융거래를 이용하기 위해 금융회사 또는 전자금융업자와 이용자가 체결하는 계약을 말합니다. 전자금융거래를 이용하기 위한 조건과 책임 등을 명시하는 계약입니다.
Q: 전자적 장치에는 어떤 것들이 포함되나요?
A: 전자적 장치는 전자금융거래정보를 전자적 방법으로 전송하거나 처리하는 데 이용되는 장치를 말합니다. 예를 들어, 현금자동지급기(ATM), 자동입출금기(CD), 지급용 단말기, 컴퓨터, 전화기 등이 전자적 장치에 포함됩니다.
Q: 접근매체란 무엇인가요?
A: 접근매체는 전자금융거래에서 거래지시를 하거나 이용자 및 거래내용의 진실성과 정확성을 확보하기 위해 사용되는 수단 또는 정보를 말합니다. 예를 들어, 전자식 카드, 전자서명, 인증서 등이 접근매체에 포함됩니다.


  4%|▍         | 3/75 [00:12<04:51,  4.05s/it]

...

Q: **제21조의3제1항은 무엇에 대한 내용인가요?**
A: **제21조의3제1항은 전자금융기반시설의 취약점을 분석ㆍ평가하도록 규정하고 있습니다.**
Q: **전자금융기반시설의 취약점 분석ㆍ평가를 하지 않으면 어떤 처벌을 받나요?**
A: **제21조의3제1항을 위반하여 전자금융기반시설의 취약점을 분석ㆍ평가하지 않으면 3년 이하의 징역 또는 3천만원 이하의 벌금에 처해집니다.**
Q: **제21조의3제2항은 어떤 내용을 규정하고 있나요?**
A: **제21조의3제2항은 전자금융기반시설의 취약점 분석ㆍ평가 결과를 바탕으로 보완조치의 이행계획을 수립ㆍ시행하도록 규정하고 있습니다.**
Q: **보완조치의 이행계획을 수립ㆍ시행하지 않으면 어떤 처벌을 받나요?**
A: **제21조의3제2항을 위반하여 보완조치의 이행계획을 수립ㆍ시행하지 않으면 2년 이하의 징역 또는 2천만원 이하의 벌금에 처해집니다.**
Q: **제22조제2항은 어떤 내용을 규정하고 있나요?**
A: **제22조제2항은 전자금융거래기록을 일정 기간 보관하도록 규정하고 있으며, 이를 위반하여 파기할 경우 처벌 대상이 됩니다.**


 99%|█████████▊| 74/75 [05:36<00:04,  4.28s/it]

Q: 전자금융거래법 위반 시 부과되는 과태료는 얼마인가요?
A: 전자금융거래법 위반 시 부과되는 과태료는 1천만원 이하입니다.
Q: 전자금융거래법 제40조제6항 위반 시 누구에게 과태료가 부과될 수 있나요?
A: 전자금융거래법 제40조제6항을 위반하여 제3자에게 재위탁을 한 자에게 과태료가 부과될 수 있습니다.
Q: 전자금융거래법 제28조제4항에 따라 해당 규정을 준용하는 선불전자지급수단을 발행하는 자는 누구인가요?
A: 전자금융거래법 제28조제4항에 따라 해당 규정을 준용하는 선불전자지급수단을 발행하는 자는 제40조제3항 제1호, 제6호부터 제8호까지 및 제10호에 해당하는 자입니다.
Q: 전자금융거래법 위반 시 과태료는 누가 부과하고 징수하나요?
A: 전자금융거래법 위반 시 과태료는 대통령령으로 정하는 바에 따라 금융위원회가 부과하고 징수합니다.
Q: 전자금융거래법은 언제 개정되었나요?
A: 전자금융거래법은 2017년 4월 18일과 2013년 5월 22일에 개정되었으며, 시행일은 2024년 9월 15일입니다.


100%|██████████| 75/75 [05:40<00:00,  4.54s/it]

Q: 이 지문은 어떤 법률의 개정 내용을 다루고 있나요?
A: 이 지문은 **전자금융거래법**의 개정 내용을 다루고 있습니다.
Q: 전자금융거래법 개정의 시행일은 언제인가요?
A: 전자금융거래법 개정은 **2024년 9월 15일**부터 시행됩니다.
Q: 개정된 전자금융거래법에서 "전자서명법"의 어떤 부분이 변경되었나요?
A: 개정된 전자금융거래법에서는 **전자서명법 제2조 제2호에 따른 전자서명**의 정의가 변경되었습니다.
Q: 개정 전 전자금융거래법 제20조는 어떤 조항을 참조했나요?
A: 개정 전 전자금융거래법 제20조는 **전자서명법 제20조**를 참조했습니다.
Q: 개정된 전자금융거래법 제20조는 어떤 조항을 참조하나요?
A: 개정된 전자금융거래법 제20조는 **전자서명법 제18조**를 참조합니다.
